# Examination of effecient NHL roster design

In [134]:
import sys
import os
import pandas as pd
import numpy as np
import datetime, time
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.formula.api import ols
from pylab import hist, show
import scipy

# import data frames

In [135]:
dm = pd.read_csv('t_play_by_play_o.csv')
dm = dm.sort_values(['Season', 'GameNumber', 'Period', 'EventNumber'], ascending=[True, True, True, True])
dm = dm [['Season', 'GameNumber', 'EventNumber', 'Period', 'AdvantageType', 'EventTimeFromZero', 'EventTimeFromTwenty', 'EventType', 'EventDetail', 'VPlayer1', 'VPlayer1Position', 'VPlayer2', 'VPlayer2Position', 'VPlayer3', 'VPlayer3Position', 'VPlayer4', 'VPlayer4Position', 'VPlayer5', 'VPlayer5Position', 'VPlayer6', 'VPlayer6Position', 'HPlayer1', 'HPlayer1Position', 'HPlayer2', 'HPlayer2Position', 'HPlayer3', 'HPlayer3Position', 'HPlayer4', 'HPlayer4Position', 'HPlayer5', 'HPlayer5Position', 'HPlayer6', 'HPlayer6Position']]

In [136]:
dg = pd.read_csv('t_play_by_play_giveaway_detail_o.csv')
dg = dg.rename(columns={'GivePlayerNumber': 'PlayerNumber', 'GivePlayerLName': 'PlayerName', 'GiveTeamCode': 'TeamCode'})

In [137]:
dt = pd.read_csv('t_play_by_play_takeaway_detail_o.csv')
dt = dt.sort_values(['Season', 'GameNumber', 'EventNumber'], ascending=[True, True, True])
dt = dt.rename(columns={'TakePlayerNumber': 'PlayerNumber', 'TakePlayerLName': 'PlayerName', 'TakeTeamCode': 'TeamCode'})

In [138]:
dl = pd.read_csv('t_play_by_play_goal_detail_o.csv')
dl = dl.sort_values(['Season', 'GameNumber', 'EventNumber'], ascending=[True, True, True])
dl = dl.rename(columns={'PlayerLName': 'PlayerName'})

In [139]:
ds = pd.read_csv('t_play_by_play_shot_detail_o.csv')
ds = ds.sort_values(['Season', 'GameNumber', 'EventNumber'], ascending=[True, True, True])
ds = ds.rename(columns={'PlayerLName': 'PlayerName'})

In [140]:
dn = pd.read_csv('t_play_by_play_miss_detail_o.csv')
dn = dn.sort_values(['Season', 'GameNumber', 'EventNumber'], ascending=[True, True, True])
dn = dn.rename(columns={'PlayerLName': 'PlayerName'})

In [141]:
db = pd.read_csv('t_play_by_play_block_detail_o.csv')
db = db.sort_values(['Season', 'GameNumber', 'EventNumber'], ascending=[True, True, True])
db = db.rename(columns={'ShotPlayerLName': 'ShotPlayerName', 'BlockPlayerLName': 'PlayerName', 'BlockPlayerNumber': 'PlayerNumber', 'BlockTeamCode': 'TeamCode'})

In [142]:
dh = pd.read_csv('t_play_by_play_hit_detail_o.csv')
dh = dh.sort_values(['Season', 'GameNumber', 'EventNumber'], ascending=[True, True, True])
dh = dh.rename(columns={'HitterPlayerLName': 'HitterPlayerName', 'HitteePlayerLName': 'HitteePlayerName'})

In [143]:
df = pd.read_csv('t_play_by_play_faceoff_detail_o.csv')
df = df.sort_values(['Season', 'GameNumber', 'EventNumber'], ascending=[True, True, True])
df = df.rename(columns={'VPlayerLName': 'VPlayerName', 'HPlayerLName': 'HPlayerName'})

In [144]:
dp = pd.read_csv('t_play_by_play_penalty_detail_o.csv')
dp = dp.sort_values(['Season', 'GameNumber', 'EventNumber'], ascending=[True, True, True])
dp = dp.rename(columns={'PenaltyPlayerLName': 'PenaltyPlayerName', 'DrawnByPlayerLName': 'DrawnByPlayerName'})

In [145]:
dd = pd.read_csv('t_game_detail_o.csv')

In [146]:
dc = pd.read_csv('t_scoring_detail_o.csv')
dc['GoalTime'] = dc['Time'].copy()
dc = dc.rename(columns={'Time': 'EventTimeFromZero', 'GoalType': 'AdvantageType'})
dc = dc [['Season', 'GameNumber', 'GoalNumber', 'Period', 'GoalTime', 'EventTimeFromZero', 'AdvantageType', 'TeamCode', 'Assist1Player', 'Assist2Player']]

In [147]:
dc

,Season,GameNumber,GoalNumber,Period,GoalTime,EventTimeFromZero,AdvantageType,TeamCode,Assist1Player,Assist2Player
0,2010,20001,1,1,402,402,EV,TOR,3 D.PHANEUF(1),NaN
1,2010,20001,2,1,537,537,EV,TOR,42 T.BOZAK(1),8 M.KOMISAREK(1)
2,2010,20001,3,1,739,739,EV,MTL,unassisted,NaN
3,2010,20001,4,3,96,96,EV,TOR,41 N.KULEMIN(1),84 M.GRABOVSKI(1)
4,2010,20001,5,3,148,148,EV,MTL,40 M.LAPIERRE(1),57 B.POULIOT(1)
5,2010,20002,1,2,171,171,PP,PHI,18 M.RICHARDS(1),22 V.LEINO(1)
6,2010,20002,2,2,1035,1035,EV,PHI,36 D.POWE(1),21 J.VAN RIEMSDYK(1)
7,2010,20002,3,3,44,44,EV,PIT,7 P.MARTIN(1),58 K.LETANG(1)
8,2010,20002,4,3,295,295,SH,PHI,unassisted,NaN
9,2010,20002,5,3,314,314,PP,PIT,4 Z.MICHALEK(1),14 C.KUNITZ(1)


# merge giveaways and takeaways 

In [148]:
dg = dg.merge(dt, on=['Season', 'GameNumber', 'EventNumber', 'TeamCode', 'PlayerName', 'PlayerNumber', 'Zone'], how='outer')
dg.sort_values(['Season', 'GameNumber', 'EventNumber'], ascending=[True, True, True], inplace=True)

In [149]:
dg

,GameNumber,EventNumber,TeamCode,PlayerNumber,PlayerName,Zone,Season
0,20001,6,TOR,35,GIGUERE,D,2010
1,20001,12,MTL,52,DARCHE,N,2010
2,20001,18,TOR,22,BEAUCHEMIN,D,2010
3,20001,20,MTL,76,SUBBAN,D,2010
4,20001,32,MTL,45,PICARD,D,2010
21164,20001,41,TOR,9,ARMSTRONG,O,2010
21165,20001,45,MTL,11,GOMEZ,D,2010
21166,20001,56,MTL,81,ELLER,N,2010
5,20001,60,TOR,35,GIGUERE,D,2010
6,20001,62,MTL,26,GORGES,N,2010


# merge goal and shot data frames

In [150]:
dl = dl.merge(ds, on=['Season', 'GameNumber', 'EventNumber', 'TeamCode', 'PlayerName', 'PlayerNumber', 'Zone', 'ShotType', 'Length'], how='outer')
dl.sort_values(['Season', 'GameNumber', 'EventNumber'], ascending=[True, True, True], inplace=True)


In [151]:
dl

,GameNumber,EventNumber,TeamCode,PlayerNumber,PlayerName,ShotType,Zone,Length,Season
7527,20001,8,MTL,81,ELLER,Wrist,O,11,2010
7528,20001,9,MTL,46,KOSTITSYN,Snap,O,8,2010
7529,20001,29,TOR,81,KESSEL,Wrist,O,25,2010
7530,20001,30,TOR,42,BOZAK,Wrist,O,12,2010
7531,20001,33,TOR,8,KOMISAREK,Wrist,O,55,2010
0,20001,35,TOR,37,BRENT,Tip-In,O,22,2010
7532,20001,42,TOR,3,PHANEUF,Snap,O,51,2010
1,20001,49,TOR,81,KESSEL,Wrist,O,14,2010
7533,20001,59,TOR,9,ARMSTRONG,Wrist,O,31,2010
7534,20001,66,MTL,17,BOYD,Wrap-around,O,13,2010


# merge miss to goal and shot

In [152]:
dl = dl.merge(dn, on=['Season', 'GameNumber', 'EventNumber', 'TeamCode', 'PlayerName', 'PlayerNumber', 'Zone', 'ShotType', 'Length'], how='outer')
dl.sort_values(['Season', 'GameNumber', 'EventNumber'], ascending=[True, True, True], inplace=True)

In [153]:
dl

,GameNumber,EventNumber,TeamCode,PlayerNumber,PlayerName,ShotType,Zone,Length,Season,ShotResult
0,20001,8,MTL,81,ELLER,Wrist,O,11,2010,NaN
1,20001,9,MTL,46,KOSTITSYN,Snap,O,8,2010,NaN
81160,20001,11,MTL,76,SUBBAN,Slap,O,58,2010,Wide of Net
2,20001,29,TOR,81,KESSEL,Wrist,O,25,2010,NaN
3,20001,30,TOR,42,BOZAK,Wrist,O,12,2010,NaN
4,20001,33,TOR,8,KOMISAREK,Wrist,O,55,2010,NaN
5,20001,35,TOR,37,BRENT,Tip-In,O,22,2010,NaN
81161,20001,40,MTL,26,GORGES,Wrist,O,26,2010,Wide of Net
6,20001,42,TOR,3,PHANEUF,Snap,O,51,2010,NaN
81162,20001,44,TOR,16,MACARTHUR,Wrist,O,50,2010,Wide of Net


# merge block to goal, shot and miss

In [154]:
dl = dl.merge(db, on=['Season', 'GameNumber', 'EventNumber', 'TeamCode', 'PlayerName', 'PlayerNumber', 'Zone', 'ShotType'], how='outer')
dl.sort_values(['Season', 'GameNumber', 'EventNumber'], ascending=[True, True, True], inplace=True)

In [155]:
dl

,GameNumber,EventNumber,TeamCode,PlayerNumber,PlayerName,ShotType,Zone,Length,Season,ShotResult,ShotTeamCode,ShotPlayerNumber,ShotPlayerName
112347,20001,7,TOR,2,SCHENN,Wrist,D,NaN,2010,NaN,MTL,76.0,SUBBAN
0,20001,8,MTL,81,ELLER,Wrist,O,11.0,2010,NaN,NaN,NaN,NaN
1,20001,9,MTL,46,KOSTITSYN,Snap,O,8.0,2010,NaN,NaN,NaN,NaN
112348,20001,10,TOR,32,VERSTEEG,Slap,D,NaN,2010,NaN,MTL,76.0,SUBBAN
2,20001,11,MTL,76,SUBBAN,Slap,O,58.0,2010,Wide of Net,NaN,NaN,NaN
112349,20001,13,MTL,26,GORGES,Slap,D,NaN,2010,NaN,TOR,16.0,MACARTHUR
112350,20001,22,TOR,22,BEAUCHEMIN,Tip-In,D,NaN,2010,NaN,MTL,81.0,ELLER
112351,20001,23,TOR,3,PHANEUF,Wrist,D,NaN,2010,NaN,MTL,14.0,PLEKANEC
112352,20001,27,MTL,26,GORGES,Wrist,D,NaN,2010,NaN,TOR,84.0,GRABOVSKI
3,20001,29,TOR,81,KESSEL,Wrist,O,25.0,2010,NaN,NaN,NaN,NaN


# merge faceoffs and hit

In [156]:
df = df.merge(dh, on=['Season', 'GameNumber', 'EventNumber', 'Zone'], how='outer')
df.sort_values(['Season', 'GameNumber', 'EventNumber'], ascending=[True, True, True], inplace=True)

In [157]:
df

,GameNumber,EventNumber,WinTeamCode,Zone,VTeamCode,VPlayerNumber,VPlayerName,HTeamCode,HPlayerNumber,HPlayerName,Season,HitterTeamCode,HitterPlayerNumber,HitterPlayerName,HitteeTeamCode,HitteePlayerNumber,HitteePlayerName
0,20001,1,MTL,N,MTL,11.0,GOMEZ,TOR,37.0,BRENT,2010,NaN,NaN,NaN,NaN,NaN,NaN
76822,20001,3,NaN,O,NaN,NaN,NaN,NaN,NaN,NaN,2010,TOR,37.0,BRENT,MTL,26.0,GORGES
76823,20001,4,NaN,O,NaN,NaN,NaN,NaN,NaN,NaN,2010,MTL,14.0,PLEKANEC,TOR,2.0,SCHENN
76824,20001,5,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,2010,MTL,76.0,SUBBAN,TOR,15.0,KABERLE
76825,20001,14,NaN,D,NaN,NaN,NaN,NaN,NaN,NaN,2010,MTL,75.0,GILL,TOR,16.0,MACARTHUR
76826,20001,15,NaN,O,NaN,NaN,NaN,NaN,NaN,NaN,2010,MTL,40.0,LAPIERRE,TOR,36.0,GUNNARSSON
1,20001,17,MTL,O,MTL,17.0,BOYD,TOR,84.0,GRABOVSKI,2010,NaN,NaN,NaN,NaN,NaN,NaN
76827,20001,19,NaN,O,NaN,NaN,NaN,NaN,NaN,NaN,2010,MTL,57.0,POULIOT,TOR,11.0,SJOSTROM
76828,20001,21,NaN,O,NaN,NaN,NaN,NaN,NaN,NaN,2010,MTL,81.0,ELLER,TOR,3.0,PHANEUF
2,20001,25,TOR,D,MTL,11.0,GOMEZ,TOR,84.0,GRABOVSKI,2010,NaN,NaN,NaN,NaN,NaN,NaN


# merge penalty to faceoff and hit

In [158]:
df = df.merge(dp, on=['Season', 'GameNumber', 'EventNumber', 'Zone'], how='outer')
df.sort_values(['Season', 'GameNumber', 'EventNumber'], ascending=[True, True, True], inplace=True)

In [159]:
df

,GameNumber,EventNumber,WinTeamCode,Zone,VTeamCode,VPlayerNumber,VPlayerName,HTeamCode,HPlayerNumber,HPlayerName,...,HitteeTeamCode,HitteePlayerNumber,HitteePlayerName,PenaltyTeamCode,PenaltyPlayerNumber,PenaltyPlayerName,PenaltyType,DrawnByTeamCode,DrawnByPlayerNumber,DrawnByPlayerName
0,20001,1,MTL,N,MTL,11.0,GOMEZ,TOR,37.0,BRENT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20001,3,NaN,O,NaN,NaN,NaN,NaN,NaN,NaN,...,MTL,26.0,GORGES,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20001,4,NaN,O,NaN,NaN,NaN,NaN,NaN,NaN,...,TOR,2.0,SCHENN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20001,5,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,...,TOR,15.0,KABERLE,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20001,14,NaN,D,NaN,NaN,NaN,NaN,NaN,NaN,...,TOR,16.0,MACARTHUR,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,20001,15,NaN,O,NaN,NaN,NaN,NaN,NaN,NaN,...,TOR,36.0,GUNNARSSON,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,20001,17,MTL,O,MTL,17.0,BOYD,TOR,84.0,GRABOVSKI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,20001,19,NaN,O,NaN,NaN,NaN,NaN,NaN,NaN,...,TOR,11.0,SJOSTROM,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,20001,21,NaN,O,NaN,NaN,NaN,NaN,NaN,NaN,...,TOR,3.0,PHANEUF,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,20001,25,TOR,D,MTL,11.0,GOMEZ,TOR,84.0,GRABOVSKI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# merge giveaway, takeaway (dg) to goal, shot, miss, block (dl)

In [160]:
dl = dl.merge(dg, on=['Season', 'GameNumber', 'EventNumber', 'TeamCode', 'PlayerName', 'PlayerNumber', 'Zone'], how='outer')
dl.sort_values(['Season', 'GameNumber', 'EventNumber'], ascending=[True, True, True], inplace=True)


In [161]:
dl.columns

Index(['GameNumber', 'EventNumber', 'TeamCode', 'PlayerNumber', 'PlayerName',
       'ShotType', 'Zone', 'Length', 'Season', 'ShotResult', 'ShotTeamCode',
       'ShotPlayerNumber', 'ShotPlayerName'],
      dtype='object')

# merge faceoff, hit, penalty (df) to goal, shot, miss, block (dl)

In [162]:
dl = dl.merge(df, on=['Season', 'GameNumber', 'EventNumber', 'Zone'], how='outer')
dl.sort_values(['Season', 'GameNumber', 'EventNumber'], ascending=[True, True, True], inplace=True)


In [163]:
dl

,GameNumber,EventNumber,TeamCode,PlayerNumber,PlayerName,ShotType,Zone,Length,Season,ShotResult,...,HitteeTeamCode,HitteePlayerNumber,HitteePlayerName,PenaltyTeamCode,PenaltyPlayerNumber,PenaltyPlayerName,PenaltyType,DrawnByTeamCode,DrawnByPlayerNumber,DrawnByPlayerName
190206,20001,1,NaN,NaN,NaN,NaN,N,NaN,2010,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
190207,20001,3,NaN,NaN,NaN,NaN,O,NaN,2010,NaN,...,MTL,26.0,GORGES,NaN,NaN,NaN,NaN,NaN,NaN,NaN
190208,20001,4,NaN,NaN,NaN,NaN,O,NaN,2010,NaN,...,TOR,2.0,SCHENN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
190209,20001,5,NaN,NaN,NaN,NaN,N,NaN,2010,NaN,...,TOR,15.0,KABERLE,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,20001,6,TOR,35.0,GIGUERE,NaN,D,NaN,2010,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20001,7,TOR,2.0,SCHENN,Wrist,D,NaN,2010,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20001,8,MTL,81.0,ELLER,Wrist,O,11.0,2010,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20001,9,MTL,46.0,KOSTITSYN,Snap,O,8.0,2010,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20001,10,TOR,32.0,VERSTEEG,Slap,D,NaN,2010,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,20001,11,MTL,76.0,SUBBAN,Slap,O,58.0,2010,Wide of Net,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# merge detail play (dd) on all ice events (dl)

In [164]:
dl = dl.merge(dd, on=['Season', 'GameNumber', 'VTeamCode', 'HTeamCode'], how='outer')
dl.sort_values(['Season', 'GameNumber', 'EventNumber'], ascending=[True, True, True], inplace=True)

In [165]:
dl

,GameNumber,EventNumber,TeamCode,PlayerNumber,PlayerName,ShotType,Zone,Length,Season,ShotResult,...,HitteePlayerNumber,HitteePlayerName,PenaltyTeamCode,PenaltyPlayerNumber,PenaltyPlayerName,PenaltyType,DrawnByTeamCode,DrawnByPlayerNumber,DrawnByPlayerName,GameDate
0,20001,1,NaN,NaN,NaN,NaN,N,NaN,2010,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010-10-07
43,20001,3,NaN,NaN,NaN,NaN,O,NaN,2010,NaN,...,26.0,GORGES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44,20001,4,NaN,NaN,NaN,NaN,O,NaN,2010,NaN,...,2.0,SCHENN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45,20001,5,NaN,NaN,NaN,NaN,N,NaN,2010,NaN,...,15.0,KABERLE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46,20001,6,TOR,35.0,GIGUERE,NaN,D,NaN,2010,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47,20001,7,TOR,2.0,SCHENN,Wrist,D,NaN,2010,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48,20001,8,MTL,81.0,ELLER,Wrist,O,11.0,2010,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49,20001,9,MTL,46.0,KOSTITSYN,Snap,O,8.0,2010,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50,20001,10,TOR,32.0,VERSTEEG,Slap,D,NaN,2010,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51,20001,11,MTL,76.0,SUBBAN,Slap,O,58.0,2010,Wide of Net,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# merge all ice events (dl) to play by play (dm)

In [166]:
dm = dm.merge(dl, on=['Season', 'GameNumber', 'EventNumber'], how='outer')
dm.sort_values(['Season', 'GameNumber', 'EventNumber'], ascending=[True, True, True], inplace=True)

In [167]:
dm.columns

Index(['Season', 'GameNumber', 'EventNumber', 'Period', 'AdvantageType',
       'EventTimeFromZero', 'EventTimeFromTwenty', 'EventType', 'EventDetail',
       'VPlayer1', 'VPlayer1Position', 'VPlayer2', 'VPlayer2Position',
       'VPlayer3', 'VPlayer3Position', 'VPlayer4', 'VPlayer4Position',
       'VPlayer5', 'VPlayer5Position', 'VPlayer6', 'VPlayer6Position',
       'HPlayer1', 'HPlayer1Position', 'HPlayer2', 'HPlayer2Position',
       'HPlayer3', 'HPlayer3Position', 'HPlayer4', 'HPlayer4Position',
       'HPlayer5', 'HPlayer5Position', 'HPlayer6', 'HPlayer6Position',
       'TeamCode', 'PlayerNumber', 'PlayerName', 'ShotType', 'Zone', 'Length',
       'ShotResult', 'ShotTeamCode', 'ShotPlayerNumber', 'ShotPlayerName',
       'WinTeamCode', 'VTeamCode', 'VPlayerNumber', 'VPlayerName', 'HTeamCode',
       'HPlayerNumber', 'HPlayerName', 'HitterTeamCode', 'HitterPlayerNumber',
       'HitterPlayerName', 'HitteeTeamCode', 'HitteePlayerNumber',
       'HitteePlayerName', 'PenaltyTeamCod

# merge scoring detail (dc) to play by play (dm)

In [168]:
dm = dm.merge(dc, on=['Season', 'GameNumber', 'Period', 'EventTimeFromZero', 'AdvantageType', 'TeamCode'], how='outer')

# remove irrelevant observations


In [169]:
dm = dm[dm['EventType']!='STOP']

# man-advantage scenarios

In [170]:
value_list = ['PP', 'SH']
dm[dm['AdvantageType'].isin(value_list)]
dm = dm[dm['AdvantageType'] != 'PP']
dm = dm[dm['AdvantageType'] != 'SH']
dm['AdvantageType'] = dm['AdvantageType'].fillna('EV')

# keep first two games of the season

In [171]:
dm = dm[dm['GameNumber'] <= 20002]

In [172]:
dm

,Season,GameNumber,EventNumber,Period,AdvantageType,EventTimeFromZero,EventTimeFromTwenty,EventType,EventDetail,VPlayer1,...,PenaltyPlayerName,PenaltyType,DrawnByTeamCode,DrawnByPlayerNumber,DrawnByPlayerName,GameDate,GoalNumber,GoalTime,Assist1Player,Assist2Player
0,2010,20001,1.0,1,EV,0,1200.0,FAC,MTL won Neu. Zone - MTL #11 GOMEZ vs TOR #37 B...,11.0,...,NaN,NaN,NaN,NaN,NaN,2010-10-07,NaN,NaN,NaN,NaN
1,2010,20001,3.0,1,EV,15,1185.0,HIT,"TOR #37 BRENT HIT MTL #26 GORGES, Off. Zone",11.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2010,20001,4.0,1,EV,46,1154.0,HIT,"MTL #14 PLEKANEC HIT TOR #2 SCHENN, Off. Zone",14.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2010,20001,5.0,1,EV,57,1143.0,HIT,"MTL #76 SUBBAN HIT TOR #15 KABERLE, Neu. Zone",14.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2010,20001,6.0,1,EV,69,1131.0,GIVE,"TOR&nbsp;GIVEAWAY - #35 GIGUERE, Def. Zone",14.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2010,20001,7.0,1,EV,73,1127.0,BLOCK,"MTL #76 SUBBAN BLOCKED BY TOR #2 SCHENN, Wris...",14.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2010,20001,8.0,1,EV,86,1114.0,SHOT,"MTL ONGOAL - #81 ELLER, Wrist, Off. Zone, 11 ft.",14.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2010,20001,9.0,1,EV,91,1109.0,SHOT,"MTL ONGOAL - #46 KOSTITSYN, Snap, Off. Zone, 8...",14.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2010,20001,10.0,1,EV,95,1105.0,BLOCK,"MTL #76 SUBBAN BLOCKED BY TOR #32 VERSTEEG, S...",14.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2010,20001,11.0,1,EV,102,1098.0,MISS,"MTL #76 SUBBAN, Slap, Wide of Net, Off. Zone, ...",15.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [173]:
dm.to_csv('pbpmerge.csv', index='False', sep=',')